In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import cenpy as cen
import seaborn as sns
import matplotlib.pyplot as plt
from shapely import wkt
import pysal as ps

%matplotlib inline

/anaconda3/envs/OSMNX/lib/python3.6/site-packages/pysal/__init__.py:65: VisibleDeprecationWarning: PySAL's API will be changed on 2018-12-31. The last release made with this API is version 1.14.4. A preview of the next API version is provided in the `pysal` 2.0 prelease candidate. The API changes and a guide on how to change imports is provided at https://pysal.org/about
  ), VisibleDeprecationWarning)


In [13]:
ls

Census Imports.ipynb     Shapefile imports.ipynb


In [3]:
#import all California census tracts...
tracts = gpd.read_file("data/tl_2017_06_tract/tl_2017_06_tract.shp")

In [4]:
## connect to ACSF survey API
dataset = 'ACSSF5Y2013'
#cen.explorer.explain(dataset)
con = cen.base.Connection(dataset)
con

Connection to 2009-2013 American Community Survey 5-Year Estimates (ID: http://api.census.gov/data/id/ACSSF5Y2013)

In [5]:
# extract all population data for census tracts in California
g_unit = 'tract:*'
g_filter = {'state':'6'}

cols = con.varslike('B01003')
cols.extend(['NAME', 'GEOID'])

data = con.query(cols, geo_unit=g_unit, geo_filter=g_filter)

data.head()

/anaconda3/envs/OSMNX/lib/python3.6/site-packages/cenpy/remote.py:172: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  df[cols] = df[cols].convert_objects(convert_numeric=convert_numeric)


,B01003_001E,B01003_001M,NAME,GEOID,state,county,tract
0,3353,332,"Census Tract 4001, Alameda County, California",14000US06001400100,06,001,400100
1,1944,147,"Census Tract 4002, Alameda County, California",14000US06001400200,06,001,400200
2,5376,539,"Census Tract 4003, Alameda County, California",14000US06001400300,06,001,400300
3,4152,346,"Census Tract 4004, Alameda County, California",14000US06001400400,06,001,400400
4,3618,422,"Census Tract 4005, Alameda County, California",14000US06001400500,06,001,400500


In [6]:
# clean up the GEOID field used for merging datasets
data['GEOID'] = data['GEOID'].astype(str)
data['GEOID'] = data['GEOID'].apply(lambda x: x.split('US')[1])

In [7]:
#merge the files...
demographics = pd.merge(data, tracts, on='GEOID')
alameda_dm = demographics[demographics.COUNTYFP == '001']

demographics.to_csv('all_demographics.csv')